In [1]:
import pandas as pd 
import numpy as np 
from tensorflow import keras 
import tensorflow as tf 
from tensorflow.keras import regularizers
import tensorboard
tensorboard.__version__
from sklearn.metrics import r2_score
import pydot
from tensorflow.keras.layers import Activation
import threading
import time 

### Read DVs 

In [2]:
# read dvs 
DVs = pd.read_csv('./Input/DVs.csv')

In [3]:
len(DVs.columns)

15

### Read IVs

In [4]:
All_IVs_plus_demo = pd.read_csv('./Input/All_IVs_plus_demo.csv')
len(All_IVs_plus_demo.columns)

68

### Read fb_Content 

In [5]:
fb_content = pd.read_csv('./Input/fb_content.csv')
len(fb_content.columns)

21

In [6]:
# New DNN Model with a sub model 
def build_and_compile_dnn_model(input_shape):
    # other_variables_num = len(All_IVs_without_reasons_to_use_FB.columns) # assume the number of variables is 67 here. In the real program, the number of input variables will be calcuated automatically. 
    # sub model 
    fb_content_input = keras.Input(shape=(21,), name='fb_content')
    fb_content_hidden_layer = keras.layers.Dense(10, activation='relu')(fb_content_input)
    fb_content_output = keras.layers.Dense(5, name='sub_output_layer', activation='softmax')(fb_content_hidden_layer)
    # other part of model 
    fb_others_input = keras.Input(shape=(input_shape,))
    fb_concatted_input = tf.keras.layers.Concatenate()([fb_content_output, fb_others_input]) # concat two types of input together 
    fb_concatted_hidden_layer1 = keras.layers.Dense(50, input_dim=2, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activation=tf.nn.relu)(fb_concatted_input)
    fb_concatted_hidden_1_dropout = keras.layers.Dropout(.2, name='hidden1_Dropout')(fb_concatted_hidden_layer1)
    fb_concatted_hidden_layer2 = keras.layers.Dense(50, input_dim=2, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activation=tf.nn.relu)(fb_concatted_hidden_1_dropout)
    fb_concatted_hidden_2_dropout = keras.layers.Dropout(.2, name='hidden2_Dropout')(fb_concatted_hidden_layer2)
    fb_concatted_output = keras.layers.Dense(15, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(fb_concatted_hidden_2_dropout)
    model = tf.keras.Model(inputs=[fb_content_input, fb_others_input], outputs=fb_concatted_output)
    learn_rate = 0.001
    opt = keras.optimizers.Adam(learning_rate=learn_rate)
    model.compile(loss='mean_absolute_error', optimizer=opt)
    return model

In [7]:
# read dvs 
DVs = pd.read_csv('./Input/DVs.csv')

# BIG 5
extraversion = DVs['extraversion']
agreeableness = DVs['agreeableness']
conscientiousness = DVs['conscientiousness']
neuroticism = DVs['neuroticism']
openness = DVs['openness']

# Other Dvs 
implusiveness = DVs['implusiveness']
rational = DVs['rational']
experiential = DVs['experiential']
normative_influence = DVs['normative.influence']
informational_influence = DVs['informational.influence']

# Potential DVs
need_for_closure = DVs['need_for_closure']
utilitarian = DVs['utilitarian']
hedonic = DVs['hedonic']
individual = DVs['individual']
collective = DVs['collective']

# create a list to store DVs
DVs_list = [extraversion, agreeableness, conscientiousness, neuroticism, openness, implusiveness, rational, experiential,normative_influence, informational_influence, need_for_closure, utilitarian, hedonic, individual,collective ]
DVs_list_name = ["extraversion", "agreeableness", "conscientiousness", "neuroticism", "openness", "implusiveness", "rational", "experiential","normative_influence", "informational_influence", "need_for_closure", "utilitarian", "hedonic", "individual","collective"]


In [8]:
# get the average score from a list 
def average(lst): 
    return sum(lst) / len(lst) 

In [9]:
input_val = [fb_content, All_IVs_plus_demo]
for k in range(0, 1):
    r2_list = []
    dnn_model = build_and_compile_dnn_model(len(All_IVs_plus_demo.columns))
    dnn_model.fit(input_val, DVs, validation_split=0.2, verbose=0, epochs=10000)
    print(r2_score(DVs, dnn_model.predict(input_val)))
    r2_ave_result = pd.DataFrame(columns = DVs_list_name)
    r2_result = []
    result = dnn_model.predict(input_val)
    for k in range(0, 15):
        dv_list = []
        for i in range(0, 597):
            dv_list.append(result[i][k])
        r2_result.append(r2_score(DVs_list[k], np.array(dv_list)))
all_r2_ave_list_series = pd.Series(r2_result, index = r2_ave_result.columns)
r2_ave_result = r2_ave_result.append(all_r2_ave_list_series, ignore_index = True)
today = time.strftime("%Y-%m-%d")
filename = 'r2_new'
ave_file = today + filename + '.csv'
r2_ave_result.to_csv(ave_file)

0.4918056745733551


In [11]:
r2_score(DVs_list[0], np.array(dv_list))

-6.121280269245354

In [ ]:
print(r2_score(DVs, dnn_model.predict(input_val)))

In [ ]:
r2_result

In [ ]:
np.array(dv_list)

In [12]:
dnn_model.save("dnn_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: dnn_model/assets


In [13]:
ls dnn_model

assets/         saved_model.pb  variables/


In [14]:
load_model = keras.models.load_model('dnn_model')

In [ ]:
load_model.summary()

In [ ]:
load_model.predict(input_val)

### Manually input FB content data into the sub-model in the saved model, check the output layer of the sub-model

In [ ]:
sub_model = tf.keras.Model(load_model.get_layer(index = 0).output, load_model.get_layer(index = 2).output)
inp = sub_model.predict(fb_content)
sub_layer = tf.keras.layers.Softmax()
sub_layer_result = pd.DataFrame(sub_layer(inp).numpy())
today = time.strftime('%y-%m-%d-%H-%M-%S')
sub_layer_file_name = today + '_sub_model_ouput.csv'
sub_layer_result.to_csv(sub_layer_file_name)